# ERA5-Land Explorer

In [ ]:
import fsspec
import xarray as xr
import hvplot.xarray

In [ ]:
# turn off hvplot warnings about NaNs
import warnings
warnings.filterwarnings("ignore")

In [ ]:
s3_json = 's3://esip-qhub/usgs/era5_land/archive3.json'

In [ ]:
%%time
fs_s3 = fsspec.filesystem("reference", fo=s3_json, ref_storage_args={'skip_instance_cache':True},
                       remote_protocol='s3', remote_options={'requester_pays':True})

In [ ]:
fs_s3.ls('')

In [ ]:
m = fs_s3.get_mapper('')
ds = xr.open_dataset(m, engine="zarr", backend_kwargs={'consolidated':False}, chunks={'time':112*10})
ds

In [ ]:
ds.sd.hvplot(x='longitude', y='latitude', geo=True, cmap='turbo', rasterize=True,  tiles='ESRI')

In [ ]:
ds.sd

In [ ]:
%%time
da = ds['sd'].sel(time=slice('1994-08-01','1996-08-01')).sel(latitude=39.6273874, longitude=-105.87, method='nearest')

In [ ]:
da

In [ ]:
%%time
da = da.load()

In [ ]:
da.hvplot(x='time', grid=True)

#### Use Intake to improve user experience

In [ ]:
import intake

In [ ]:
intake_catalog_url = 's3://esip-qhub/usgs/era5_land/era5_intake.yml'

In [ ]:
cat = intake.open_catalog(intake_catalog_url)
list(cat)

In [ ]:
ds = cat['era5-land'].to_dask()

In [ ]:
ds.sd

#### Use Panel to improve user experience

In [ ]:
import panel as pn
from geoviews import tile_sources as gvts
import pandas as pd
import cf_xarray

In [ ]:
datetimes = pd.to_datetime(ds['time'])
start = datetimes.min(); end = datetimes.max()
enabled = datetimes.strftime("%Y-%m-%d %H:%M:%S").tolist()
print(start)

In [ ]:
datetime_picker = pn.widgets.DatetimePicker(name="Datetime", value=start, start=start, end=end, enabled_dates=enabled)

In [ ]:
base_map_select = pn.widgets.Select(name='Basemap:', options=gvts.tile_sources, value=gvts.OSM)

In [ ]:
ds.data_vars

In [ ]:
vars = list(ds.data_vars)
var_select = pn.widgets.Select(name='ERA5-Land Variables:', options=vars, value=vars[0])

In [ ]:
def set_tools(plot, element):
    plot.state.toolbar.active_inspect = None   #turns off hover by default

In [ ]:
# quadmesh with DatePicker
@pn.depends(var_select, base_map_select, datetime_picker)
def qm_dp_plot(var, base_map, date):
    da = ds[var].cf.sel(T=date, method="nearest")
    mesh = da.hvplot.image(x=da.cf['longitude'].name, y=da.cf['latitude'].name, 
                            rasterize=True, geo=True, title=var, 
                            cmap='turbo').opts(alpha=0.7, hooks=[set_tools], active_tools=['pan', 'box_zoom'])
    viz = pn.panel(mesh * base_map)
    return viz

In [ ]:
dashboard = pn.Column(pn.Row(base_map_select, var_select, datetime_picker), qm_dp_plot)

In [ ]:
dashboard.servable('ERA5-Land Explorer')